## This notebook is part of the Apache Spark training delivered by CERN IT


Run this notebook from Jupyter with Python kernel
- When running on CERN SWAN, do not attach the notebook to a Spark cluster, but rather run it locally on the SWAN container (which is the default)
- If running this outside CERN SWAN, please make sure to have PySpark installed: `pip install pyspark`


In order to run this notebook as slides:
 - on SWAN click on the button "Enter/Exit RISE slideshow" in the ribbon
 - on other environments please make sure to have the RISE extension installed `pip install RISE`

### SPARK DataFrame Hands-On Lab
Contact: Luca.Canali@cern.ch

### Objective: Perform Basic DataFrame Operations
1. Creating DataFrames
2. Select columns
3. Add, rename and drop columns
4. Filtering rows
5. Aggregations

## Getting started: create the SparkSession

In [1]:
!pip install pyspark

In [2]:
# !pip install pyspark

from pyspark.sql import SparkSession

spark = (SparkSession.builder
          .master("local[*]") \
          .appName("DataFrame HandsOn 1") \
          .config("spark.ui.showConsoleProgress","false") \
          .getOrCreate()
        )

spark

The master `local[*]` means that the executors are in the same node that is running the driver. The `*` tells Spark to start as many executors as there are logical cores available

### Hands-On 1 - Construct a DataFrame from csv file
This demostrates how to read a csv file and construct a DataFrame.  
We will use the online retail dataset from Kaggle, credits: https://www.kaggle.com/datasets/vijayuv/onlineretail


#### First, let's inspect the csv content

In [3]:
#modify below code to use the downloaded dataset

!gzip -cd ../data/online-retail-dataset.csv.gz 2>&1| head -n3

gzip: ../data/online-retail-dataset.csv.gz: No such file or directory


In [4]:
online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"

In [12]:
df = (spark.read
        .option("header", "true")
        .option("timestampFormat", "M/d/yyyy H:m")
        .csv("/content/OnlineRetail.csv",
             schema=online_retail_schema)
     )

#### Inspect the data

In [14]:
df.show(10, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerId|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850     |United Kingdom|
|536365   |22752

#### Show columns

In [15]:
df.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Country: string (nullable = true)



### Hands-On 2 - Spark Transformations - select, add, rename and drop columns

Select dataframe columns

In [16]:
# select single column

df.select("Country").show(2)

+--------------+
|       Country|
+--------------+
|United Kingdom|
|United Kingdom|
+--------------+
only showing top 2 rows



Select multiple columns


In [17]:
df.select("StockCode","Description","UnitPrice").show(n=2, truncate=False)

+---------+----------------------------------+---------+
|StockCode|Description                       |UnitPrice|
+---------+----------------------------------+---------+
|85123A   |WHITE HANGING HEART T-LIGHT HOLDER|2.55     |
|71053    |WHITE METAL LANTERN               |3.39     |
+---------+----------------------------------+---------+
only showing top 2 rows



In [18]:
df.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerId',
 'Country']

In [19]:
# select first 5 columns
df.select(df.columns[0:5]).show(2)

+---------+---------+--------------------+--------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|
+---------+---------+--------------------+--------+-------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|
+---------+---------+--------------------+--------+-------------------+
only showing top 2 rows



In [20]:
# selects all the original columns and adds a new column that specifies high value item
(df.selectExpr(
   "*", # all original columns
   "(UnitPrice > 100) as HighValueItem")
   .show(2)
)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|HighValueItem|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|        false|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|        false|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-------------+
only showing top 2 rows



In [21]:
# selects all the original columns and adds a new column that specifies high value item
(df.selectExpr(
  "sum(Quantity) as TotalQuantity",
  "cast(sum(UnitPrice) as int) as InventoryValue")
  .show()
)

+-------------+--------------+
|TotalQuantity|InventoryValue|
+-------------+--------------+
|      1985439|       1069499|
+-------------+--------------+



#### Adding, renaming and dropping columns

In [23]:
# add a new column called InvoiceValue
from pyspark.sql.functions import expr
df_1 = (df
        .withColumn("InvoiceValue", expr("UnitPrice * Quantity"))
        .select("InvoiceNo","Description","UnitPrice","Quantity","InvoiceValue")
       )
df_1.show(5, False)

# rename InvoiceValue to LineTotal
df_2 = df_1.withColumnRenamed("InvoiceValue","LineTotal")
df_2.show(5, False)

# drop a column
df_2.drop("LineTotal").show(5, False)

+---------+-----------------------------------+---------+--------+------------+
|InvoiceNo|Description                        |UnitPrice|Quantity|InvoiceValue|
+---------+-----------------------------------+---------+--------+------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |2.55     |6       |15.299999   |
|536365   |WHITE METAL LANTERN                |3.39     |6       |20.34       |
|536365   |CREAM CUPID HEARTS COAT HANGER     |2.75     |8       |22.0        |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|3.39     |6       |20.34       |
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |3.39     |6       |20.34       |
+---------+-----------------------------------+---------+--------+------------+
only showing top 5 rows

+---------+-----------------------------------+---------+--------+---------+
|InvoiceNo|Description                        |UnitPrice|Quantity|LineTotal|
+---------+-----------------------------------+---------+--------+---------+
|536365   |WHITE HANGING

### Hands-On 3 - Spark Transformations - filter, sort and cast

In [24]:
from pyspark.sql.functions import col

# select invoice lines with quantity > 50 and unitprice > 20
df.where(col("Quantity") > 20).where(col("UnitPrice") > 50).show(2)
df.filter(df.Quantity > 20).filter(df.UnitPrice > 50).show(2)
df.filter("Quantity > 20 and UnitPrice > 50").show(2)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerId|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerId|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerId|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------

In [25]:
# select invoice lines with quantity > 100 or unitprice > 20
df.where((col("Quantity") > 100) | (col("UnitPrice") > 20)).show(2)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536378|    21212|PACK OF 72 RETROS...|     120|2010-12-01 09:37:00|     0.42|     14688|United Kingdom|
|     NULL|        D|            Discount|      -1|2010-12-01 09:41:00|     27.5|     14527|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [26]:
from pyspark.sql.functions import desc, asc

# sort in the default order: ascending
df.orderBy(expr("UnitPrice")).show(2)

df.orderBy(col("Quantity").desc(), col("UnitPrice").asc()).show(10)

+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|   547593|    20785|       NULL|      -2|2011-03-24 11:18:00|      0.0|      NULL|United Kingdom|
|   536414|    22139|       NULL|      56|2010-12-01 11:52:00|      0.0|      NULL|United Kingdom|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   541431|    23166|MEDIUM CERAMIC TO...|   74215|2011-0

### Hands-On 4 - Spark Transformations - aggregations
full list of built int functions - https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions

In [27]:
%%time
# Count distinct customers
from pyspark.sql.functions import countDistinct
df.select(countDistinct("CustomerID")).show()

+--------------------------+
|count(DISTINCT CustomerID)|
+--------------------------+
|                      2794|
+--------------------------+

CPU times: user 7.31 ms, sys: 2.03 ms, total: 9.34 ms
Wall time: 1.25 s


In [28]:
%%time
# approx. distinct stock items
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("CustomerID", 0.1)).show()

+---------------------------------+
|approx_count_distinct(CustomerID)|
+---------------------------------+
|                             2408|
+---------------------------------+

CPU times: user 5.47 ms, sys: 592 µs, total: 6.07 ms
Wall time: 944 ms


In [29]:
# average, maximum and minimum purchase quantity
from pyspark.sql.functions import avg, max, min
( df.select(
    avg("Quantity").alias("avg_purchases"),
    max("Quantity").alias("max_purchases"),
    min("Quantity").alias("min_purchases"))
   .show()
)

+-----------------+-------------+-------------+
|    avg_purchases|max_purchases|min_purchases|
+-----------------+-------------+-------------+
|9.311866839261594|        74215|       -74215|
+-----------------+-------------+-------------+



### Hands-On 5 - Spark Transformations - grouping and windows

In [30]:
# count of items on the invoice
df.groupBy("InvoiceNo", "CustomerId").count().show(5)

# grouping with expressions
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
  .show(5)

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536573|     17025|    4|
|   537228|     17677|    1|
|   537419|     13495|   14|
|   538093|     12682|   33|
|   538648|     17937|    5|
+---------+----------+-----+
only showing top 5 rows

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536532| 25.36986301369863|  16.850272831671976|
|   537632|               1.0|                 0.0|
|   538708| 10.61111111111111|   7.150282736359209|
|   538877|14.258278145695364|   27.56989037543246|
|   538993| 9.333333333333334|   2.748737083745107|
+---------+------------------+--------------------+
only showing top 5 rows



### Read the csv file into DataFrame

`%%time` is an iPython magic https://ipython.readthedocs.io/en/stable/interactive/magics.html


It's possible to read files without specifying the schema. Some file formats (Parquet is one of them) include the schema, which means that Spark can start reading the file. For format without schema (csv, json...) Spark can infer the schema. Let's see what's the difference in terms of time and of results:

In [31]:
online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"

In [33]:
%%time
df = spark.read \
        .option("header", "true") \
        .option("timestampFormat", "M/d/yyyy H:m")\
        .csv("/content/OnlineRetail.csv",
             schema=online_retail_schema)

CPU times: user 2.44 ms, sys: 0 ns, total: 2.44 ms
Wall time: 32.5 ms


In [34]:
%%time
df_infer = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("/content/OnlineRetail.csv")

CPU times: user 7.27 ms, sys: 2.6 ms, total: 9.88 ms
Wall time: 2.11 s


## Exercises

Reminder: documentation at
https://spark.apache.org/docs/latest/api/python/index.html

If you didn't run the previous cells, run the following one:

In [36]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("DataFrame HandsOn 1") \
        .config("spark.ui.showConsoleProgress","false") \
        .getOrCreate()

online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"

df = spark.read \
        .option("header", "true") \
        .option("timestampFormat", "M/d/yyyy H:m")\
        .csv("/content/OnlineRetail.csv",
             schema=online_retail_schema)

Task: Show 5 lines of the "description" column

In [37]:
df.select("Description").show(5)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|CREAM CUPID HEART...|
|KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|
+--------------------+
only showing top 5 rows



Task: Count the number of distinct invoices in the dataframe

In [38]:
df.select("InvoiceNo").distinct().count()

22062

Task: Find out in which month most invoices have been issued

In [39]:
import pyspark.sql.functions as F
from pyspark.sql.functions import desc, month
df.select(month(df.InvoiceDate).alias("Month")).groupBy("Month").count().orderBy(desc("count")).show(1)

+-----+-----+
|Month|count|
+-----+-----+
|   11|84711|
+-----+-----+
only showing top 1 row



Task: Filter the lines where the Quantity is more than 30

In [ ]:
df.select(['*']).where(df.Quantity > 30).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536367|    84879|ASSORTED COLOUR B...|      32|2010-12-01 08:34:00|     1.69|     13047|United Kingdom|
|   536370|    10002|INFLATABLE POLITI...|      48|2010-12-01 08:45:00|     0.85|     12583|        France|
|   536370|    22492|MINI PAINT SET VI...|      36|2010-12-01 08:45:00|     0.65|     12583|        France|
|   536371|    22086|PAPER CHAIN KIT 5...|      80|2010-12-01 09:00:00|     2.55|     13748|United Kingdom|
|   536374|    21258|VICTORIAN SEWING ...|      32|2010-12-01 09:09:00|    10.95|     15100|United Kingdom|
|   536376|    22114|HOT WATER BOTTLE ...|      48|2010-12-01 09:32:00|     3.45|     15291|United Kingdom|
|   536376|    21733|RED HAN

Task: Show the four most sold items (by quantity)

In [43]:
df.groupBy("Description").sum('Quantity').orderBy(desc('sum(Quantity)')).show(4)

+--------------------+-------------+
|         Description|sum(Quantity)|
+--------------------+-------------+
|WORLD WAR 2 GLIDE...|        53847|
|JUMBO BAG RED RET...|        47363|
|ASSORTED COLOUR B...|        36381|
|      POPCORN HOLDER|        36334|
+--------------------+-------------+
only showing top 4 rows



Bonus question: why do these two operations return different results? Hint: look at the documentation

In [44]:
print(df.select("InvoiceNo").distinct().count())
from pyspark.sql.functions import countDistinct
df.select(countDistinct("InvoiceNo")).show()

22062
+-------------------------+
|count(DISTINCT InvoiceNo)|
+-------------------------+
|                    22061|
+-------------------------+





The two operations, `df.select("InvoiceNo").distinct().count()` and `df.select(countDistinct("InvoiceNo")).show()`, return different results due to how they handle distinct counts and the underlying execution in Spark.

1. **`df.select("InvoiceNo").distinct().count()`**: This operation first selects the `InvoiceNo` column, then applies the `distinct()` transformation to remove duplicate invoice numbers.  Finally, the `count()` action calculates the number of unique `InvoiceNo` values.  This is a *two-stage* operation.  It materializes the distinct set of invoice numbers in memory before performing the count.

2. **`df.select(countDistinct("InvoiceNo")).show()`**: This operation uses the `countDistinct()` function which is a built-in aggregate function.  It directly calculates the approximate count of distinct invoice numbers within a single stage.  Critically, the `approx_count_distinct` function provides an *approximation* of the count, using a smaller memory footprint by utilizing a probabilistic data structure (HyperLogLog). The `show()` action then displays the result.  This is a *one-stage* operation.


Therefore, the difference in results stems from the fact `countDistinct` is an *approximation* of the true distinct count while `distinct().count()` calculates the exact distinct count. If the number of distinct elements is very large, then `approx_count_distinct` will often give a similar but slightly different result compared to the exact calculation, which has been computed by `distinct().count()`.  Refer to the documentation for `approx_count_distinct` for more details on the approximation method and its accuracy.


# Lab 4 Submission
**Group 7**: 24280069, 24280052  
**Member**: Syed Muhammad Mujtaba, Usmar Haider